<a href="https://colab.research.google.com/github/alazarchuk/AI-Toolbox/blob/main/Monthly_Reporting_(Phi_4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [ ]:
import torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)


In [ ]:
model_path = "microsoft/phi-4"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4")
summarization_model = AutoModelForCausalLM.from_pretrained(
          model_path, quantization_config=bnb_config, device_map="auto"
)

In [ ]:
pipe = pipeline(
    "text-generation",
    model=summarization_model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 1000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

In [ ]:
from datetime import datetime

In [ ]:
date_start = datetime.fromisoformat("2025-05-01")
date_end = datetime.fromisoformat("2025-06-01")
author_filter = "alazarchuk"

In [ ]:
print(date_start)
print(date_end)

In [ ]:
!pip install -q -U PyGithub

In [ ]:
from google.colab import userdata
from github import Github
from github import Auth
from collections import defaultdict

access_token = userdata.get('GITHUB_TOKEN')
github_organizations = userdata.get('GITHUB_ORGANIZATIONS').split(",")

auth = Auth.Token(access_token)

g = Github(auth=auth)
author_filter = g.get_user().login

commits_per_repo = defaultdict(list)

for org_name in github_organizations:
  org = g.get_organization(org_name)
  for repo in org.get_repos():
    commits = repo.get_commits(since=date_start, until=date_end, author=author_filter)
    try:
      count = commits.totalCount
    except:
      count = 0
    if count > 0:
      for commit in commits:
        commits_per_repo[org.name + " -> " + repo.name].append(commit.commit.message)


In [ ]:
for repo in commits_per_repo.keys():
  commits = commits_per_repo[repo]

  summary_messages = [
      {
          "role": "user",
          "content": "Write one sentence summary in past tense on the work done based on commit messages. " +
                     "Keep it not more then 20 words. " +
                     "Exclude mentions of pull requests, commits and merges. " +
                     "Commits: " + "\n".join(commits)}
  ]

  output = pipe(summary_messages, **generation_args)
  print("="*60)
  print(repo)
  print("-"*60)
  summary = output[0]['generated_text']
  print(summary)
  print("-"*60)
  translate_messages = [
        {"role": "assistant", "content": "I am usefull assistant that can translate summary of developemt work from english to ukranian"},
        {"role": "user", "content": "Fixed issues related to seeds and restoring files"},
        {"role": "assistant", "content": "Виправив помилки пов'язаних із сідами та відновлення файлів"},
        {"role": "user", "content": "Added additional fields to Address"},
        {"role": "assistant", "content": "Додав додаткові поля до Адреси"},
        {"role": "user", "content": "Refactored exception handling, enhanced code documentation and updated dependencies"},
        {"role": "assistant", "content": "Відрефакторив обробку помилок, покращив документацію коду і оновив залежності"},
        {"role": "user", "content": summary}
  ]
  output = pipe(translate_messages, **generation_args)
  translated_summary = output[0]['generated_text']
  print(translated_summary)
  print("="*60)